In [643]:
import pandas as pd 
import numpy as np
import math

In [644]:
res = pd.read_csv("results.csv")
res.rename(columns={'Unnamed: 0':'State'}, inplace=True)
res = res.iloc[:-1]

In [645]:
vader = pd.read_csv("statewise_total_idea1.csv")
vader.rename(columns={'Unnamed: 0':'State'}, inplace=True)

In [646]:
bnb = pd.read_csv("statewise_total_idea2.csv")
bnb.rename(columns={'Unnamed: 0':'State'}, inplace=True)

In [647]:
vader_subj = pd.read_csv("statewise_total_idea3.csv")
vader_subj.rename(columns={'Unnamed: 0':'State'}, inplace=True)

In [648]:
bnb_subj = pd.read_csv("statewise_total_idea4.csv")
bnb_subj.rename(columns={'Unnamed: 0':'State'}, inplace=True)

In [649]:
vader_bnb = pd.read_csv("statewise_total_idea5.csv")
vader_bnb.rename(columns={'Unnamed: 0':'State'}, inplace=True)

In [650]:
sentiment = "vader_bnb"
if sentiment=="bnb":
    df = bnb
elif sentiment=="vader":
    df = vader
elif sentiment=="vader_subj":
    df = vader_subj
elif sentiment=="bnb_subj":
    df = bnb_subj
elif sentiment=="vader_bnb":
    df = vader_bnb
df['Actual Biden Percentage'] = res['Biden Percentage']
df['Actual Trump Percentage'] = res['Trump Percentage']
df['Actual Winner'] = res["Winner"]
df["Total Tweets"] = df["Trump Total count"] + df["Biden Total count"]

In [651]:
df.head()

,State,Trump Positive count,Trump Negative count,Trump Total count,Biden Positive count,Biden Negative count,Biden Total count,Actual Biden Percentage,Actual Trump Percentage,Actual Winner,Total Tweets
0,Alabama,237,90,327,227,85,312,0.37,0.62,Trump,639
1,Alaska,112,34,146,119,50,169,0.43,0.53,Trump,315
2,Arizona,797,362,1159,926,295,1221,0.49,0.49,Biden,2380
3,Arkansas,132,88,220,154,44,198,0.35,0.62,Trump,418
4,California,7440,3847,11287,8103,2018,10121,0.63,0.34,Biden,21408


In [652]:
df.describe()

,Trump Positive count,Trump Negative count,Trump Total count,Biden Positive count,Biden Negative count,Biden Total count,Actual Biden Percentage,Actual Trump Percentage,Total Tweets
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.00000
mean,866.560000,444.580000,1311.140000,888.880000,242.880000,1131.760000,0.477400,0.50080,2442.90000
std,1474.219007,737.441526,2209.551548,1573.610857,390.946817,1961.513613,0.103506,0.10325,4165.91146
min,15.000000,9.000000,24.000000,30.000000,14.000000,45.000000,0.270000,0.31000,69.00000
25%,134.500000,64.000000,203.500000,121.500000,37.500000,163.500000,0.402500,0.41250,361.00000
50%,299.000000,183.500000,482.000000,298.000000,111.000000,424.500000,0.490000,0.49500,924.50000
75%,801.500000,438.000000,1272.500000,915.500000,265.250000,1175.250000,0.557500,0.58000,2344.75000
max,7440.000000,3847.000000,11287.000000,8103.000000,2018.000000,10121.000000,0.660000,0.70000,21408.00000


In [653]:
def getPreds(method):
    if method == 1:
        df['Biden prediction'] = df['Biden Positive count']/(df['Trump Positive count'] + df['Biden Positive count'])
        df['Trump prediction'] = df['Trump Positive count']/(df['Trump Positive count'] + df['Biden Positive count'])
    elif method == 2:
        df['Biden prediction'] = df['Trump Negative count']/(df['Trump Negative count'] + df['Biden Negative count'])
        df['Trump prediction'] = df['Biden Negative count']/(df['Trump Negative count'] + df['Biden Negative count'])
    elif method == 3:
        trump_pos = df['Trump Positive count']/(df['Trump Total count'])
        biden_pos = df['Biden Positive count']/(df['Biden Total count'])
        df['Biden prediction'] = biden_pos/(trump_pos + biden_pos)
        df['Trump prediction'] = trump_pos/(trump_pos + biden_pos)
    elif method == 4:
        trump_neg = df['Biden Negative count']/(df['Biden Total count'])
        biden_neg = df['Trump Negative count']/(df['Trump Total count'])
        df['Biden prediction'] = biden_neg/(trump_neg + biden_neg)
        df['Trump prediction'] = trump_neg/(trump_neg + biden_neg)

In [654]:
def kl_divergence(p,q):
    return sum(p[i] * math.log2(p[i]/q[i]) for i in range(len(p)))

In [655]:
getPreds(3)

In [656]:
actual_biden = df['Actual Biden Percentage']
predicted_biden = df['Biden prediction']
# actual_biden = actual_biden/actual_biden.sum()
# predicted_biden = predicted_biden/predicted_biden.sum()
actual_biden/=100
predicted_biden/=100
actual_trump = df['Actual Trump Percentage']
predicted_trump = df['Trump prediction']
actual_trump/=100
predicted_trump/=100
# actual_trump = actual_trump/actual_trump.sum()
# predicted_trump = predicted_trump/predicted_trump.sum()

In [657]:
def getMetrics():
    kl_div_biden = kl_divergence(actual_biden, predicted_biden)
    kl_div_trump = kl_divergence(actual_trump, predicted_trump)
    mae_biden = np.mean(np.abs(df['Actual Biden Percentage'] - df['Biden prediction']))
    mae_trump = np.mean(np.abs(df['Actual Trump Percentage'] - df['Trump prediction']))
    print("MAE Biden: ", mae_biden)
    print("MAE Trump: ", mae_trump)
    print("KL divergence for Biden: ", kl_div_biden)
    print("KL divergence for Trump: ", kl_div_trump)

In [658]:
getMetrics()

MAE Biden:  0.0009059081795644669
MAE Trump:  0.0008507488791546231
KL divergence for Biden:  -0.03283953602104783
KL divergence for Trump:  0.03419607984802087


In [659]:
'''
MAE Biden:  0.08316090137390701
MAE Trump:  0.08154000665172986
KL divergence for Biden:  0.03208874055390086
KL divergence for Trump:  0.030404295951667742
'''

'\nMAE Biden:  0.08316090137390701\nMAE Trump:  0.08154000665172986\nKL divergence for Biden:  0.03208874055390086\nKL divergence for Trump:  0.030404295951667742\n'

In [660]:
col = df['Biden prediction'] > df['Trump prediction']
count = 0
for e in col:
    if e == True:
        count += 1
count

48

In [661]:
len(res[res['Winner']=="Biden"])

25

In [662]:
len(res[res['Winner']=="Trump"])

25